In [1]:
import pandas as pd
import numpy as np
import random
import scipy
import time

from sklearn.model_selection import train_test_split

# Function to sample queries
def sample_queries(df, frac):
    srch_ids = pd.Series(df["srch_id"].unique()).sample(frac=frac, random_state=42)
    return df.loc[df.srch_id.isin(srch_ids)]

start = time.time()

in_train = pd.read_csv("data/training_set_VU_DM_2014.csv")
in_test = pd.read_csv("data/test_set_VU_DM_2014.csv")
#in_test = pd.read_csv("data/kaggle_test.csv")

# Speed up execution
#in_train = sample_queries(in_train, 0.1)

print("Elapsed time", (time.time() - start)/60)
in_train.head()

Elapsed time 1.1884581565856933


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [2]:
start = time.time()

# Add mean, median and std per property for a variety of numericals
def add_mean_median_std_per_factor(df, factor, numerical, prefix):
    grouped = df.groupby(factor)[numerical]
    mean_by_factor = grouped.mean().to_dict()
    median_by_factor = grouped.median().to_dict()
    std_by_factor = grouped.std().to_dict()
    
    df[prefix + "_mean_" + numerical] = df[factor].map(lambda f: mean_by_factor[f])
    df[prefix + "_median_" + numerical] = df[factor].map(lambda f: median_by_factor[f])
    df[prefix + "_std_" + numerical] = df[factor].map(lambda f: std_by_factor[f])
    
for df in [in_train, in_test]:
    f = add_mean_median_std_per_factor
    f(df, "srch_id", "price_usd", "srch")
    f(df, "srch_id", "prop_starrating", "srch")
    f(df, "srch_id", "prop_review_score", "srch")
    
    f(df, "srch_destination_id", "price_usd", "dest")
    f(df, "srch_destination_id", "prop_starrating", "dest")
    f(df, "srch_destination_id", "prop_review_score", "dest")
    
    f(df, "prop_id", "price_usd", "prop")
    f(df, "prop_id", "prop_review_score", "prop")
    
print("Elapsed time", (time.time() - start)/60)
in_train.head()

Elapsed time 2.3622241020202637


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,dest_std_prop_starrating,dest_mean_prop_review_score,dest_median_prop_review_score,dest_std_prop_review_score,prop_mean_price_usd,prop_median_price_usd,prop_std_price_usd,prop_mean_prop_review_score,prop_median_prop_review_score,prop_std_prop_review_score
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0.813616,3.760563,4.0,0.7554,118.758742,118.0,17.778734,3.5,3.5,0.0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0.813616,3.760563,4.0,0.7554,152.054082,129.0,390.928573,4.0,4.0,0.0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0.813616,3.760563,4.0,0.7554,168.540871,165.0,345.479493,4.5,4.5,0.0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0.813616,3.760563,4.0,0.7554,82.598870,65.1,305.765579,4.0,4.0,0.0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0.813616,3.760563,4.0,0.7554,137.648135,117.0,433.301637,3.5,3.5,0.0


In [3]:
start = time.time()

def rank(s):
    return pd.Series(scipy.stats.rankdata(s.fillna(-1)))

# Add ranking of various features per query
for df in [in_train, in_test]:
    grouped = df.groupby("srch_id")
    df["srch_price_rank"] = grouped.price_usd.apply(rank).reset_index(drop=True)
    df["srch_star_rank"] = grouped.prop_starrating.apply(rank).reset_index(drop=True)
    df["srch_review_score_rank"] = grouped.prop_review_score.apply(rank).reset_index(drop=True)
    df["srch_location_score1_rank"] = grouped.prop_location_score1.apply(rank).reset_index(drop=True)
    df["srch_location_score2_rank"] = grouped.prop_location_score2.apply(rank).reset_index(drop=True)
    
print("Elapsed time", (time.time() - start)/60)

Elapsed time 11.601807216803232


In [4]:
start = time.time()

# Split on search ids
srch_ids = in_train["srch_id"].unique()
srch_ids_train, srch_ids_test_val = train_test_split(srch_ids, test_size=0.2, random_state = 42)
srch_ids_test, srch_ids_val = train_test_split(srch_ids_test_val, test_size=0.5, random_state = 42)

_train = in_train.loc[in_train.srch_id.isin(srch_ids_train)]
_val = in_train.loc[in_train.srch_id.isin(srch_ids_val)]
_test = in_train.loc[in_train.srch_id.isin(srch_ids_test)]

print("Elapsed time", (time.time() - start)/60)

Elapsed time 0.4060278654098511


In [5]:
#start = time.time()

# Estimate probability of being clicked and booked
#grouped = _train.groupby("prop_id")    
#click_prob_by_prop = grouped.click_bool.mean().to_dict()
#book_prob_by_prop = grouped.booking_bool.mean().to_dict()

#mean_click_prob = in_train.click_bool.mean()
#mean_book_prob = in_train.booking_bool.mean()
    
#for df in [_train, _val, _test, in_test]:
#    df["click_probability"] = df.prop_id.map(lambda p: click_prob_by_prop.setdefault(p, mean_click_prob))
#    df["booking_probability"] = df.prop_id.map(lambda p: book_prob_by_prop.setdefault(p, mean_book_prob))
    
#    df.click_probability = (df.click_probability + mean_click_prob) / 2
#    df.booking_probability = (df.booking_probability + mean_book_prob) / 2
    
#print("Elapsed time", (time.time() - start)/60)

In [6]:
start = time.time()

# Undersample a series so that there are as many non-clicked items as clicked and booked
def downsample_series(s):
    s1 = s.loc[s.click_bool == 1]
    s2 = s.loc[s.click_bool == 0]
    
    if (len(s2) == 0):
        return s1
    
    s2 = s2.sample(frac=min(1, len(s1)/len(s)))
    return s1.append(s2).sort_index()

_train_undersampled = _train.groupby("srch_id").apply(downsample_series).reset_index(drop=True)

print("Elapsed time", (time.time() - start)/60)

Elapsed time 19.237187202771505


In [7]:
start = time.time()

# Save to CSV
#_train.to_csv("data/processed/_train.csv", index=False)
_val.to_csv("data/processed/_val.csv", index=False)
_test.to_csv("data/processed/_test.csv", index=False)
_train_undersampled.to_csv("data/processed/_train_undersampled.csv", index=False)
in_test.to_csv("data/processed/test.csv", index=False)

print("Elapsed time", (time.time() - start)/60)

Elapsed time 7.31405797402064
